In [1]:
import sys
import pandas as pd
import numpy as np
from xtlearn.preprocessing import NumericBinner
from xtlearn.feature_selection import PvalueSelector
from sklearn.model_selection import train_test_split

df = pd.read_csv('/home/risknow/Datasets/heart.csv')
X = df.drop(columns="output")
y = df["output"]

# faking some nans
# X.loc[X['age'] < 40, 'age'] = np.nan

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    random_state=42,
)

In [1]:
import os
def file_names(path=None,directory=False):
    
    if path is None:
        path = os.getcwd()
    
    list_ = os.listdir(path)
    
    if directory:
        return list_
    
    else:
        return [f for f in list_ if os.path.isfile(os.path.join(path, f))]
    
file_names(directory=False)

['upload.sh',
 '.test.ipynb',
 '.gitignore',
 'README.md',
 'setup.cfg',
 'LICENSE',
 'setup.py']

In [12]:
selector = PvalueSelector()

selector.fit(X_train,y_train)

selector.transform(X_train)

,age,sex,cp,trtbps,chol,thalachh,exng,oldpeak,slp,caa,thall
144,76,0,2,140,197,116,0,1.1,1,0,2
215,43,0,0,132,341,136,1,3.0,1,0,3
230,47,1,2,108,243,152,0,0.0,2,0,2
66,51,1,2,100,222,143,1,1.2,1,0,2
288,57,1,0,110,335,143,1,3.0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...
170,56,1,2,130,256,142,1,0.6,1,1,1
59,57,0,0,128,303,159,0,0.0,2,1,2
37,54,1,2,150,232,165,0,1.6,2,0,3
116,41,1,2,130,214,168,0,2.0,1,0,2


In [2]:
# import warnings

# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", category=DeprecationWarning)
#     import md5, sha
    
binner = NumericBinner().fit(X_train,y_train)

In [3]:
binner.bins['age']

,total,0,%0,1,%1,woe,iv,min,max,mean,std,total_iv
bin,,,,,,,,,,,,
0,10,3,0.030928,7,0.060870,-0.677077,0.020273,NaN,NaN,NaN,NaN,0.414932
1,31,8,0.082474,23,0.200000,-0.885832,0.104108,40.0,44.0,42.483871,1.338431,0.414932
2,37,13,0.134021,24,0.208696,-0.442883,0.033072,45.0,51.0,48.216216,2.262436,0.414932
3,31,13,0.134021,18,0.156522,-0.155201,0.003492,52.0,55.0,53.451613,1.206613,0.414932
4,39,24,0.247423,15,0.130435,0.640225,0.074899,56.0,59.0,57.589744,1.044231,0.414932
5,27,20,0.206186,7,0.060870,1.220043,0.177292,60.0,63.0,61.407407,1.152231,0.414932
6,37,16,0.164948,21,0.182609,-0.101713,0.001796,64.0,77.0,67.189189,3.389836,0.414932


In [250]:
X.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall'],
      dtype='object')

In [225]:
eval_information_value(
    binner, y_train, y_values=[0, 1], goods=0, treat_inf=True
)

,total,0,%0,1,%1,woe,iv
feature,,,,,,,
1.0,99,34,0.361702,65,0.601852,-0.509190,0.122282
2.0,103,60,0.638298,43,0.398148,0.471981,0.113346


In [226]:
column = binner
target = y_train

In [227]:
def check_value(x, ref):
    return np.sum(np.where(x == ref, 1, 0))

data = pd.DataFrame({"feature": column, "target": target})

df = data.groupby("feature").agg(
    **{
        "total": ("target", "count"),
    }
)

# for val in y_values:

#     df = df.merge(
#         data.groupby("feature").agg(
#             **{
#                 str(val): ("target", lambda x: check_value(x, val)),
#             }
#         ),
#         left_index=True,
#         right_index=True,
#         how="left",
#     )

#     sum_val = df[str(val)].sum()
#     df["%" + str(val)] = df[str(val)] / sum_val

# def log_0(c_0, c_1):
#     if c_0 == 0:
#         return -np.inf
#     elif c_1 == 0:
#         return np.inf
#     elif (c_0 != 0) & (c_1 != 0):
#         return np.log(c_0 / c_1)

# if goods == 0:
#     df["woe"] = df.apply(lambda row: log_0(row["%0"], row["%1"]), axis=1)
#     df["iv"] = (df["%0"] - df["%1"]) * df["woe"]
# else:
#     df["woe"] = df.apply(lambda row: log_0(row["%0"], row["%1"]), axis=1)
#     df["iv"] = (df["%1"] - df["%0"]) * df["woe"]

# if treat_inf:
#     df = treat_iv_infinity(df)


In [231]:
data.groupby("feature").agg(
    **{
        "total": ("target", "count"),
    }
)

,total
feature,
1.0,99
2.0,103


In [170]:
bins = {}
iv_temp = 0
i = 4
column_name = "age"
X_copy = X_train.copy()
X_local = X_copy.copy()

X_temp = X_local.loc[X_local[column_name].notna()][[column_name]].copy()
n = len(X_temp)
n_quant = 1000 if n >= 1000 else n

binner = Pipeline(
    steps=(
        ("scaler", RobustScaler()),
        (
            "quantile",
            QuantileTransformer(output_distribution="uniform", n_quantiles=n_quant),
        ),
        (
            "binning",
            KBinsDiscretizer(n_bins=i, strategy="kmeans", encode="ordinal"),
        ),
    )
)

X_temp[column_name + "_transf"] = binner.fit_transform(X_temp).flatten() + 1
X_temp[column_name + "_transf"] = X_temp[column_name + "_transf"].astype(
    int, errors="ignore"
)
X_temp.loc[X_copy[column_name].isna(), column_name + "_transf"] = 0

summary = X_temp.groupby(column_name + "_transf").agg(
    col_min=(column_name, "min"),
    col_max=(column_name, "max"),
    col_mean=(column_name, "mean"),
    col_std=(column_name, "std"),
)


X_local.loc[X_local[column_name].notna(), column_name] = X_temp[column_name + "_transf"]

# X_local.loc[X_copy[column_name].isna(), column_name] = 0
# X_local[column_name] = X_local[column_name].astype(int, errors="ignore")

iv = eval_information_value(
    X_local[column_name], y, y_values=[0, 1], goods=0, treat_inf=True
)

iv = iv.merge(
    summary.reset_index(), how="left", right_on=column_name + "_transf", left_index=True
).set_index(column_name + "_transf")

X_local[column_name] = X_local[column_name].apply(
    lambda x: iv["woe"][x]
)

if (iv["iv"].sum() > iv_temp) & (
    np.isfinite(X_local[column_name].unique()).all()
):
    iv_temp = iv["iv"].sum()
    bin_temp = iv
    binner_temp = binner

In [83]:
binner.bins['age']

,total,0,%0,1,%1,woe,iv
feature,,,,,,,
1,31,9,0.092784,22,0.191304,-0.723597,0.071289
2,37,12,0.123711,25,0.217391,-0.563748,0.052812
3,41,16,0.164948,25,0.217391,-0.276066,0.014478
4,30,19,0.195876,11,0.095652,0.716765,0.071837
5,36,25,0.257732,11,0.095652,0.991202,0.160654
6,37,16,0.164948,21,0.182609,-0.101713,0.001796


In [16]:
eval_information_value(
    X_train['restecg'], y_train, y_values=[0, 1], goods=0, treat_inf=True
)

,total,0,%0,1,%1,woe,iv
feature,,,,,,,
0,98,51,0.525773,47,0.408696,0.251899,0.029492
1,110,43,0.443299,67,0.582609,-0.273271,0.038069
2,4,3,0.030928,1,0.008696,1.268833,0.028209
